# 동영상 뽑다가 마지막에 실패

In [105]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime

items = []
name_list = []
url = 'https://www.megabox.co.kr/on/oh/oha/Movie/selectMovieList.do'
headers = {"User-Agent": "Mozilla/5.0"}
for page_number in range(1, 3):
    payload = {
        "currentPage": str(page_number),
        "recordCountPerPage": "20",
        "pageType": "rfilmDe",
        "ibxMovieNmSearch": "",
        "onairYn": "MSC02",
        "specialType": "",
        "specialYn": "N"}
    response = requests.post(url,json=payload, headers=headers)
    data = json.loads(response.text)

    num = len(data['movieList'])
    now = datetime.now().strftime('%Y.%m.%d')
    for i in range(num):
        filter_movie = data['movieList'][i]
        start_dt = filter_movie['rfilmDe']
        name = filter_movie['movieNm']
        
        if name not in name_list:
            if start_dt > now:
                name_list.append(name)
                items.append({
                    'code' : filter_movie['movieNo'],
                    'url' : 'https://www.megabox.co.kr/movie-detail?rpstMovieNo=' + filter_movie['movieNo'],
                    'img_url' : 'https://www.megabox.co.kr/' + filter_movie['imgPathNm'],
                    'name' : name,
                    'date' : start_dt,
                    'booking' : filter_movie['boxoBokdRt'],
                })
            
            
# 동영상 뽑는거

url = 'https://www.megabox.co.kr/on/oh/oha/Movie/selectMovieStilDetail.do'
headers = {"User-Agent": "Mozilla/5.0"}

mp4_list = []

num = len(items)

for i in range(num):
    code = items[i]['code']
    payload = {
        'rpstMovieNo' : code
    }

    response = requests.post(url, json=payload, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        mp4 = soup.find('div', class_='swiper-slide').find('source')['src']
        # mp4_list.append(mp4)
        items[i]['mp4_url'] = mp4

    except:
        # mp4_list.append('0')
        items[i]['mp4_url'] = '0'

# 세부정보 뽑는거

url = 'https://www.megabox.co.kr/on/oh/oha/Movie/selectMovieInfo.do'
headers = {"User-Agent": "Mozilla/5.0"}


num = len(items)

for i in range(num):
    code = items[i]['code']
    payload = {
        'rpstMovieNo' : code
    }

    response = requests.post(url, json=payload, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    soup.find('div', class_='line')
    
    filter_soup = soup.find('div', class_='line').find_all('p')
    director = filter_soup[0].text.split('\xa0: ')[1]
    genre_runtime = filter_soup[1].text.split('\xa0: ')[1]
    
    items[i]['director'] = director
    items[i]['genre_runtime'] = genre_runtime

In [106]:
len(items)

40

In [110]:
items[39]

{'code': '24006600',
 'url': 'https://www.megabox.co.kr/movie-detail?rpstMovieNo=24006600',
 'img_url': 'https://www.megabox.co.kr//SharedImg/2024/02/06/pzLUTKyxmyhgvO3gK4AhuOUv1zO0L55J.jpg',
 'name': '오멘: 저주의 시작',
 'date': '2024.04',
 'booking': 0,
 'mp4_url': 'https://s3550.smartucc.kr/encodeFile/3550/2024/02/06/3311e966d67e7f600606912814b8498e_W.mp4',
 'director': '아르카샤 스티븐슨',
 'genre_runtime': '공포(호러) / 0 분'}

In [85]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.movies.upcoming

In [86]:
db.insert_many(items)

In [87]:
a = list(set(items))

TypeError: unhashable type: 'dict'

In [204]:
url = 'https://www.megabox.co.kr/on/oh/oha/Movie/selectMovieInfo.do'
headers = {"User-Agent": "Mozilla/5.0"}




code = items[5]['code']
payload = {
    'rpstMovieNo' : code
}

response = requests.post(url, json=payload, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

soup.find('div', class_='line')

filter_soup = soup.find('div', class_='line').find_all('p')
director = filter_soup[0].text.split('\xa0: ')[1]
genre_runtime = filter_soup[1].text.split('\xa0: ')[1]




In [205]:
director_list = {'감독': 'director', '장르': 'genre_runtime', '등급': 'age', '개봉일': 'date'}
k = []
num = len(filter_soup)
for i in range(num):
    type = filter_soup[i].text.split('\xa0: ')
    target = type[1]
    k.append({director_list[type[0]]: target})
    director_list.pop(type[0])

if len(director_list) != 0:
    for key, val in director_list.items():
        k.append({key: '0'})

for dict in k:
    for key, val in dict.items():
        if key != 'date':
            items[5][key] = val

[{'genre_runtime': '애니메이션 / 70 분'}, {'age': '전체관람가'}, {'date': '2024.02.15'}, {'감독': '0'}]


In [202]:
director = {'e' : 1}
len(director)

1

In [191]:
a = [{'genre_runtime': '공연 / 50 분'}, {'age': '전체관람가'}, {'date': '2024.02.14'}, {'0': '0'}]
director_list.pop('감독')
director_list

{'장르': '0', '등급': '0', '개봉일': '0'}